In [23]:
import githubwebstats, query_ads
import pandas as pd
import numpy as np
from tqdm import tqdm
import requests

In [2]:
github_urls = githubwebstats._clean_github_urls()

In [ ]:
results = pd.DataFrame(columns=['github_url', 'repo_name', 'metions', 'n_unq_authors', 'n_unq_first_authors',
                                'arxiv_ids', 'citation_count', 'nasa_ack_mentions', 'unq_keywords'])

for idx in tqdm(range(len(github_urls))):
    github_url = github_urls[idx]
    if requests.get('https://{}'.format(github_url)).status_code != 200:
        continue
    repo_name = github_url.split('/')[-1]
    if len(repo_name) < 4:
        continue

    r = query_ads.query_ads("full:' " + repo_name +" '")

    results.loc[idx, 'github_url'] = github_url
    results.loc[idx, 'repo_name'] = repo_name
    results.loc[idx, 'metions'] = len(r)
    results.loc[idx, 'n_unq_authors'] = len(np.unique([item for sublist in [r1['author'] for r1 in r if r1['author'] is not None] for item in sublist]))
    results.loc[idx, 'n_unq_first_authors'] = len(np.unique([r1['first_author'] for r1 in r if r1['first_author'] is not None]))


    results.loc[idx, 'unq_keywords'] = np.unique([item for sublist in [r1['keyword'] for r1 in r if r1['keyword'] is not None] for item in sublist])
    results.loc[idx, 'citation_count'] = np.sum([r1['citation_count'] for r1 in r if r1['citation_count'] is not None])
    results.loc[idx, 'nasa_ack_mentions'] = np.asarray([np.any(['NASA' in r1['ack'], 'NNX' in r1['ack']])  for r1 in r if r1['ack'] is not None]).sum()
    results.loc[idx, 'arxiv_ids'] = [r1['arxiv_id'] for r1 in r if r1['arxiv_id'] is not None]


  0%|          | 0/381 [00:00<?, ?it/s]
Exception in thread Thread-6:
Traceback (most recent call last):
  File "/Users/ch/miniconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Users/ch/miniconda3/lib/python3.6/site-packages/tqdm/_monitor.py", line 63, in run
    for instance in self.tqdm_cls._instances:
  File "/Users/ch/miniconda3/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

/Users/ch/miniconda3/lib/python3.6/site-packages/ads/search.py:544: UserWarning: Response rows did not match input rows. Setting this query's rows to 2000
  "Setting this query's rows to {}".format(self.query['rows']))
 84%|████████▎ | 319/381 [20:16<03:56,  3.81s/it]

In [ ]:
results.to_csv('repo_success_metrics.csv')

In [ ]:
results

'vis'

True